# DestinE Platform CacheA data access

## Prerequisites
### DestinE Platform Credentials

You need to have an account on the [Destination Earth Platform](https://auth.destine.eu/realms/desp/account).

#### ⚠️ Warning: Authorized Access Only
The usage of this notebook and data access is reserved only to authorized user groups.

In [1]:
!pip install requests
!pip install pystac_client

## Access the Data
With a DESP account you can access the dataset in the CacheA cache via STAC Specification

You need to import the proper packages to access the catalog via pystac

In [2]:
import requests
from pystac_client import Client
import json

In [3]:
stac_base_url = "https://cachea.destine.eu"

# Search Data by Collection
## Define the search query

You can exploit the data of a collection specifying:
- The collection name (e.g. SENTINEL-1)
- A Bounding Box (e.g. [-180, -90, 180, 90])
- A date interval in the format YYYY-MM-DD/YYYY-MM-DD (e.g. "2024-09-24/2024-09-25")

You can change the code below according to your needs

In [4]:
stac_url = f"{stac_base_url}/stac/api"
catalog = Client.open(stac_url)

search = catalog.search(method="GET",
    collections=["SENTINEL-1"],  # Sentinel-2 L2A Collection
    bbox=[-180, -90, 180, 90],  # Bounding box: [min lon, min lat, max lon, max lat]
    datetime="2024-09-24/2024-09-25",      # Date range
    limit=10                               # Limit the number of results
)

# Get the number of items

In [5]:
items = list(search.items())

print(f"Number of returned items: {len(items)}")

Number of returned items: 15


# Display the Properties of each item

In [6]:
for item in items:
    print()
    print(f" *** Properties of the item {item.properties['productIdentifier']} ***")
    print()
    for key, value in item.properties.items():
        print(f"Property : {key} - Value: {value}")
    


 *** Properties of the item S1A_IW_SLC__1SDV_20240924T044122_20240924T044149_055802_06D177_D4F5.SAFE ***

Property : productIdentifier - Value: S1A_IW_SLC__1SDV_20240924T044122_20240924T044149_055802_06D177_D4F5.SAFE
Property : datetime - Value: 2024-09-24T04:41:22.298000Z
Property : instrument - Value: SAR-C
Property : platformName - Value: SENTINEL-1
Property : productType - Value: IW_SLC__1S
Property : timeCoverageStart - Value: 2024-09-24T04:41:22.298Z
Property : timeCoverageEnd - Value: 2024-09-24T04:41:49.283Z
Property : provider - Value: ONDA
Property : providerDescription - Value: ONDA
Property : uuid - Value: bb96719e-6b1b-435e-a375-4d4a20bc82e0
Property : publicationDate - Value: 2024-09-24T19:56:40.317531Z
Property : format - Value: SAFE
Property : collection - Value: SENTINEL-1
Property : updated - Value: 2024-09-24T19:56:40.618278Z
Property : created - Value: 2024-09-24T19:46:28.190342Z
Property : resto:keywords - Value: [{'id': 'collection:SENTINEL-1', 'name': 'SENTINEL-

# Display the assets of the items

In [7]:
for item in items:
    print()
    print(f" *** Assets available for the item {item.properties['productIdentifier']} ***")
    print()
    for key, value in item.assets.items():
        print(f"{key}")


 *** Assets available for the item S1A_IW_SLC__1SDV_20240924T044122_20240924T044149_055802_06D177_D4F5.SAFE ***

Sentinel-1/SAR-C/IW_SLC__1S/2024/09/24/S1A_IW_SLC__1SDV_20240924T044122_20240924T044149_055802_06D177_D4F5.SAFE/S1A_IW_SLC__1SDV_20240924T044122_20240924T044149_055802_06D177_D4F5.SAFE-report-20240924T052550.pdf
Sentinel-1/SAR-C/IW_SLC__1S/2024/09/24/S1A_IW_SLC__1SDV_20240924T044122_20240924T044149_055802_06D177_D4F5.SAFE/annotation/calibration/calibration-s1a-iw1-slc-vh-20240924t044123-20240924t044148-055802-06d177-001.xml
Sentinel-1/SAR-C/IW_SLC__1S/2024/09/24/S1A_IW_SLC__1SDV_20240924T044122_20240924T044149_055802_06D177_D4F5.SAFE/annotation/calibration/calibration-s1a-iw1-slc-vv-20240924t044123-20240924t044148-055802-06d177-004.xml
Sentinel-1/SAR-C/IW_SLC__1S/2024/09/24/S1A_IW_SLC__1SDV_20240924T044122_20240924T044149_055802_06D177_D4F5.SAFE/annotation/calibration/calibration-s1a-iw2-slc-vh-20240924t044124-20240924t044149-055802-06d177-002.xml
Sentinel-1/SAR-C/IW_SLC__1

# Retrieve the token and use it for data download

**NOTE**: the token is valid for a limited period of time and needs to be regenerated and reconfigured periodically by running the cell above.

In [8]:
%%capture cap
%run ./get_access_token.py

Type your username :  ········
Type your password:  ········


In [9]:
output_1 = cap.stdout.split('}\n')
access_token = output_1[-1][0:-1]


# Download data

You can download an asset by specifying the asset name in the code below.
    
Please set as filepath one of the asset you want to download, getting one of the assets listed in the **Display the assets of the items** cell and as filename the output name of the downloaded file

In [10]:
filepath = "Sentinel-1/SAR-C/IW_SLC__1S/2024/09/24/S1A_IW_SLC__1SDV_20240924T044122_20240924T044149_055802_06D177_D4F5.SAFE/S1A_IW_SLC__1SDV_20240924T044122_20240924T044149_055802_06D177_D4F5.SAFE-report-20240924T052550.pdf"

filename = "S1A_IW_SLC__1SDV_20240924T044122_20240924T044149_055802_06D177_D4F5.SAFE-report-20240924T052550.pdf"

In [12]:
#download

url = f"{stac_base_url}/stac/download?filename={filepath}"

payload = {}
headers = {
  'Authorization': f'Bearer {access_token}'
}

response = requests.request("POST", url, headers=headers, data=payload)

# Check if the request was successful
if response.status_code == 200:
    # Define the file name to save the file as
    file_name = filename  
    
    # Save the file
    with open(file_name, 'wb') as file:
        file.write(response.content)
    print(f"File saved as {file_name}")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

File saved as S1A_IW_SLC__1SDV_20240924T044122_20240924T044149_055802_06D177_D4F5.SAFE-report-20240924T052550.pdf
